In [ ]:
#Removed weekends
#got the average for each hour 0-1 [Mon-fri]=B_01, 1-2 [Mon-fri]=B_12, 2-3 [Mon-fri]=B_23 ... 23-24 [Mon-fri]=B2324
#Now the data points, Lets say in 0-1 [Mon-fri] > 0.95*B_01 not an anomaly, else anomaly.  

from statistics import mean
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
from pydot import Dot, Edge, Node
from automata.fa.dfa import DFA
import numpy as np
from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import itertools
from time import time

with open('C:/Users/bchan/Desktop/TUD/gurobi/PD_dataset/train_data1.txt', "r") as my_file:
        train_pow = [float(line.strip()) for line in my_file.readlines()]
with open('C:/Users/bchan/Desktop/TUD/gurobi/PD_dataset/test_data1.txt', "r") as my_file:
        test_pow = [float(line.strip()) for line in my_file.readlines()]

sorted_data=sorted(train_pow+test_pow)

def my_scaler(column,min_val,max_val):
    range_val = sorted_data[-1]-sorted_data[0]
    scaled_values = [round(((max_val - min_val)/range_val)*(i - sorted_data[0]) + min_val,1) for i in column]
    return scaled_values

scaled_values=my_scaler(train_pow,0,1)
scaled_values1=my_scaler(test_pow,0,1)

converted_num = [str(i) for i in scaled_values]
converted_num1 = [str(i) for i in scaled_values1]
Lst=converted_num
Lst1=converted_num1
FL = [l.split(',') for l in Lst]
FL_test = [l.split(',') for l in Lst1]

uniq_list=set(Lst)
alphabet = set(item for string in uniq_list for item in string.split(",") if item != ',')

Pref_S = set()
for string in uniq_list:
    prefixes = string.split(',')
    for i in range(1, len(prefixes) + 1):
        prefix = ','.join(prefixes[:i])
        Pref_S.add(prefix)
Pref_S.add('')

def train(n):
    #Function to train a model and return a DFA
    #n denotes no of states, m denotes the ALFRED goal, lower and upper denotes the two bounds
    
    states = {str(f'q{i}') for i in range(n)}
    start_state = 'q0'

    env=gp.Env(empty=True)
    env.setParam('OutputFlag', 0)
    env.start()

    #t0 = time()
    # Creating a new model
    mdb = gp.Model("DFA_DBPD", env=env)

    t0 = time()

    #DECISION VARIABLES
    delta = mdb.addVars(states, alphabet, states, vtype=gp.GRB.BINARY, name='delta')
    x = mdb.addVars(Pref_S, states, vtype=gp.GRB.BINARY, name='x')
    f = mdb.addVars(states, vtype=gp.GRB.BINARY, name='f')
    alpha = mdb.addVars(len(Lst), states, vtype=gp.GRB.BINARY, name= 'alpha')
    LB = mdb.addVar(lb=0.10,ub=0.10,vtype=gp.GRB.CONTINUOUS, name='LB')
    UB = mdb.addVar(lb=0.11, ub=0.11,vtype=gp.GRB.CONTINUOUS, name='UB')

    #OBJECTIVE
    mdb.setObjective(1, gp.GRB.MINIMIZE)

    #AUTOMATA CONSTRAINTS
    #Constraint1
    for state0 in states:
        for symbol in alphabet:
            mdb.addConstr(sum(delta[state0,symbol,state1] for state1 in states)==1, name=f'delta[{state0},{symbol}]')
    
    #Constraint2
    for word in Pref_S:
        mdb.addConstr(sum(x[word,state1] for state1 in states)==1, name=f'x[{word}]')


    #Constraint3
    mdb.addConstr(x['',start_state]==1, name='initial_state')

    #Constraint4
    for state0, word, symbol, state1 in itertools.product(states, Pref_S, alphabet, states):
        if (word + ',' + symbol) in Pref_S:
            mdb.addConstr(x[word,state0] + delta[state0, symbol, state1] -1 <= x[word + ',' + symbol, state1], name=f'transition[{state0},{word},{symbol},{state1}]')

        if word == '' and symbol in Pref_S:
            mdb.addConstr(x[word, state0] + delta[state0, symbol, state1] - 1 <= x[symbol, state1], name=f'transition[{state0},{word},{symbol},{state1}]')


    #BOUND CONSTRAINTS
    for i, word in enumerate(Lst):
        for state1 in states:
            mdb.addConstr(alpha[i, state1] >= x[word,state1] + f[state1] -1, name=f'bound_1[{state1},{i}]')
            mdb.addConstr(alpha[i, state1] <= x[word,state1], name=f'bound_2[{state1},{i}]')
            mdb.addConstr(alpha[i, state1] <= f[state1], name=f'bound_3[{state1},{i}]')        
         
    mdb.addConstr(sum(alpha[i, state1] for i,word in enumerate(Lst) for state1 in states )/len(Lst) >= LB, name=f'lowerBound')
    mdb.addConstr(sum(alpha[i, state1] for i,word in enumerate(Lst) for state1 in states )/len(Lst) <= UB, name=f'upperBound')

    #Write the model
    mdb.write(rf'C:\Users\bchan\Desktop\TUD\Thesis\model_DB_{n}.lp')

    mdb.optimize()
    #print('Obj: %g' % mdb.ObjVal)

    t1 = time()
    #print(t1)
    print("Run time", (t1-t0), "seconds")

    if mdb.status == 1:
        status = 'LOADED'
        print(f'DFAmodel_{n}: {status}')
            
    elif mdb.status == 2:
        print(f'DFAmodel_{n} OPTIMAL')
        status='OPTIMAL'
        transitions = mdb.getAttr('X', delta)
        t_values = [(s1,a,s2) for s1 in states for s2 in states for a in alphabet if round(transitions[s1, a, s2],0) == 1]
        f_s = mdb.getAttr('X', f)
        final_state = {s1 for s1 in states if round(f_s[s1],0) == 1}

        transition_dict = create_transition_dict(states, alphabet, t_values)
        
        dfa1 = DFA(states=states,input_symbols=alphabet, transitions= transition_dict, initial_state= start_state, final_states=final_state)
        #print(f'Final_state:{final_state}')
        accepted = 0
        rejected = 0
        for w in FL:
            if dfa1.accepts_input(w):
                #print(f'{w}:accepted')
                #print('accepted')
                accepted += 1             
            else:
                rejected += 1        
        print(f'Accepted:{accepted}')
        print(f'Rejected:{rejected}')

        create_diagram(rf'C:\Users\bchan\Desktop\TUD\Thesis\diagram_DB_{n}.png', states, start_state,final_state, transition_dict)
        return dfa1    
    
    elif mdb.status == 3:
        status = 'INFEASIBLE'
        print(f'DFAmodel_{n}: {status}')
    else:
        print('status unknown, DEBUG!!')    


def create_transition_dict(states, alphabet, t_values):
    transition_dict = {}

    for state in states:
        transition_dict[state] = {}
        for symbol in alphabet:
            transition_dict[state][symbol] = None

    for trans in t_values:
        current_state, symbol, next_state = trans
        transition_dict[current_state][symbol] = next_state

    return transition_dict

def create_diagram(path, states, start_state, final_state, transition_dict):

    graph = Dot(graph_type='digraph', rankdir='LR')
    nodes = {}
    for state in states:
        if state == start_state:
            # color start state with green
            if state in final_state:
                initial_state_node = Node(
                    state,
                    style='filled',
                    peripheries=2,
                    fillcolor='#66cc33')
            else:
                initial_state_node = Node(
                    state, style='filled', fillcolor='#66cc33')
            nodes[state] = initial_state_node
            graph.add_node(initial_state_node)
        else:
            if state in final_state:
                state_node = Node(state, peripheries=2)
            else:
                state_node = Node(state)
            nodes[state] = state_node
            graph.add_node(state_node)
    # adding edges
    for from_state, lookup in transition_dict.items():
        for to_label, to_state in lookup.items():
            graph.add_edge(Edge(
                nodes[from_state],
                nodes[to_state],
                label=to_label
            ))
    if path:
        graph.write_png(path)
    return graph

def test( dfa1, FL_test):
    with open('C:/Users/bchan/Desktop/TUD/gurobi/PD_dataset/test_label1.txt', "r") as my_file:
        test_label = [int(line.strip()) for line in my_file.readlines()]
    accepted = 0
    rejected = 0
    Predicted_labels=[]
    for w in FL_test:
        if dfa1.accepts_input(w):
            Predicted_labels.append(1)
            accepted += 1             
        else:
            Predicted_labels.append(0)
            rejected += 1        
    print(f'Accepted in Testing:{accepted}')
    print(f'Rejected in Testing:{rejected}')    

    accuracy = accuracy_score(test_label, Predicted_labels)
    print(f'Accuracy:{round(accuracy,2)}')
    f1 = f1_score(test_label, Predicted_labels, average='binary', pos_label=1)
    print(f'F1_score:{round(f1,2)}\n')

for n in range(2,3):
    dfa1= train(n)
    test(dfa1, FL_test)